# 문제 06 유형
- Dataset_06.csv 이용

In [1]:
import pandas as pd

df1_orig = pd.read_csv('datasets/Dataset_06.csv', encoding='utf8')

df1 = df1_orig.copy()

df1.head(3)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,sqft_living15,sqft_lot15
0,2597000006,20150309T000000,347500.0,3,1.50,1180,8353,1.0,0,0,3,7,1180,0,1960,0,98155,1710,8748
1,8813400345,20150414T000000,575000.0,2,1.00,980,3663,1.0,0,0,5,7,980,0,1909,0,98105,1620,3706
2,226059106,20150102T000000,489500.0,3,1.75,2090,65558,1.0,0,0,3,8,1330,760,1977,0,98072,2450,47178


In [2]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4323 entries, 0 to 4322
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             4323 non-null   int64  
 1   date           4323 non-null   object 
 2   price          4323 non-null   float64
 3   bedrooms       4323 non-null   int64  
 4   bathrooms      4323 non-null   float64
 5   sqft_living    4323 non-null   int64  
 6   sqft_lot       4323 non-null   int64  
 7   floors         4323 non-null   float64
 8   waterfront     4323 non-null   int64  
 9   view           4323 non-null   int64  
 10  condition      4323 non-null   int64  
 11  grade          4323 non-null   int64  
 12  sqft_above     4323 non-null   int64  
 13  sqft_basement  4323 non-null   int64  
 14  yr_built       4323 non-null   int64  
 15  yr_renovated   4323 non-null   int64  
 16  zipcode        4323 non-null   int64  
 17  sqft_living15  4323 non-null   int64  
 18  sqft_lot

## 1. warterfront에 따라 평균 주택 가격 계산
- 평균 가격 차이의 절대값, 소수점 이하는 버리고 정수만 기술

In [3]:
df1.waterfront.unique()

array([0, 1], dtype=int64)

In [4]:
df1_wf = df1[df1['waterfront'] == 1]
df1_nwf = df1[df1['waterfront'] == 0]

av_price_wf = df1_wf.mean()['price']
av_price_nwf = df1_nwf.mean()['price']

diff_av_price = abs(av_price_wf - av_price_nwf)
print(diff_av_price)

1167272.748018648


### 답
1167272

## 2. 7개 변수 간의 상관 분석
- price, bedrooms, bathrooms, sqft_living, sqft_lot, floors, yr_built
- price와의 상관계수의 절댓값이 가장 큰 변수와 작은 변수

In [8]:
df1_corr_var = ['price', 'bedrooms', 'bathrooms', 'sqft_living',
               'sqft_lot', 'floors', 'yr_built']

df1[df1_corr_var].corr().abs()['price'].sort_values(ascending=False)

price          1.000000
sqft_living    0.709157
bathrooms      0.534277
bedrooms       0.279824
floors         0.270423
sqft_lot       0.106341
yr_built       0.059895
Name: price, dtype: float64

### 답
sqft_living, yr_built

### 참고

In [19]:
df1[df1_corr_var].corr().abs()['price'][1:].nlargest(1)

sqft_living    0.709157
Name: price, dtype: float64

In [20]:
df1[df1_corr_var].corr().abs()['price'][1:].nsmallest(1)

yr_built    0.059895
Name: price, dtype: float64

## 3. price를 종속변수로 하여 회귀분석
- id, date, zipcode를 제외한 나머지를 독립변수로
- 통계적 유의성을 갖지 못하는 독립변수를 제거하면 남는 변수
- 제거 후 음의 회귀계수를 가지는 변수 개수

In [10]:
from statsmodels.formula.api import ols

drop_cols = ['id', 'date', 'zipcode', 'price']
df1_features = df1.drop(columns=drop_cols).columns

formula = 'price ~ ' + '+'.join(df1_features)
formula

'price ~ bedrooms+bathrooms+sqft_living+sqft_lot+floors+waterfront+view+condition+grade+sqft_above+sqft_basement+yr_built+yr_renovated+sqft_living15+sqft_lot15'

In [11]:
result1 = ols(formula, df1).fit()
result1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.681
Model:                            OLS   Adj. R-squared:                  0.680
Method:                 Least Squares   F-statistic:                     656.0
Date:                Tue, 24 Aug 2021   Prob (F-statistic):               0.00
Time:                        10:22:02   Log-Likelihood:                -58960.
No. Observations:                4323   AIC:                         1.180e+05
Df Residuals:                    4308   BIC:                         1.180e+05
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept      6.302e+06   2.95e+05     21.401      0.000    5.73e+06    6.88e+06
bedrooms        -2.5e+04   3680.510     -6.791      0.000   -3.22e+04   -1.78e+04
bathrooms      5.179e+04   7163.408      7.230      0.000    3.77e+04    6.58e+04
sqft_living      92.3081      5.074     18.192      0.000      82.360     102.256
sqft_lot          0.0054      0.093      0.059      0.953      -0.176       0.187
floors         3.748e+04   8141.392      4.604      0.000    2.15e+04    5.34e+04
waterfront     6.277e+05   3.91e+04     16.064      0.000    5.51e+05    7.04e+05
view            4.04e+04   4797.344      8.421      0.000     3.1e+04    4.98e+04
condition      1.571e+04   5317.387      2.954      0.003    5280.918    2.61e+04
grade          1.306e+05   4700.604     27.789      0.000    1.21e+05     1.4e+05
sqft_above       33.7358      4.981      6.773      0.000      23.971      43.501
sqft_basement    58.5722      5.840     10.030      0.000      47.123      70.021
yr_built      -3666.8957    150.805    -24.316      0.000   -3962.551   -3371.241
yr_renovated     17.7032      8.265      2.142      0.032       1.499      33.908
sqft_living15    23.0114      7.561      3.044      0.002       8.188      37.834
sqft_lot15       -0.2323      0.142     -1.637      0.102      -0.511       0.046
==============================================================================
Omnibus:                     2170.721   Durbin-Watson:                   2.011
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            38362.875
Skew:                           1.971   Prob(JB):                         0.00
Kurtosis:                      17.051   Cond. No.                     3.29e+17
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.5e-22. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

### 답
13, 2

### 참고
- p-value가 0.05보다 작은 변수들

In [22]:
result1.pvalues[result1.pvalues < 0.05]

Intercept         1.204633e-96
bedrooms          1.262643e-11
bathrooms         5.681393e-13
sqft_living       2.616908e-71
floors            4.266372e-06
waterfront        1.931929e-56
view              5.028272e-17
condition         3.157318e-03
grade            1.760208e-156
sqft_above        1.431354e-11
sqft_basement     2.030032e-23
yr_built         1.717539e-122
yr_renovated      3.226128e-02
sqft_living15     2.352216e-03
dtype: float64

In [23]:
len(result1.pvalues[result1.pvalues < 0.05])-1

13

In [37]:
sum(result1.pvalues < 0.05) - 1

13

# 문제 07 유형
- DataSet_07.csv 이용

In [12]:
import pandas as pd

df2_orig = pd.read_csv('datasets/Dataset_07.csv', encoding='utf8')
df2 = df2_orig.copy()

df2.head(3)

,Serial_No,GRE,TOEFL,University_Rating,SOP,LOR,CGPA,Research,Chance_of_Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72


In [13]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Serial_No          400 non-null    int64  
 1   GRE                400 non-null    int64  
 2   TOEFL              400 non-null    int64  
 3   University_Rating  400 non-null    int64  
 4   SOP                400 non-null    float64
 5   LOR                400 non-null    float64
 6   CGPA               400 non-null    float64
 7   Research           400 non-null    int64  
 8   Chance_of_Admit    400 non-null    float64
dtypes: float64(4), int64(5)
memory usage: 28.2 KB


## 1. 합격 가능성에 가장 영향이 큰 점수
- GRE, TOEFL, CGPA
- 피어슨 상관계수
- Chance_of_Admit와의 가장 큰 상관계수를 소수점 넷째 자리에서 반올림하여 셋째 자리까지

In [25]:
corr2 = df2[['GRE', 'TOEFL', 'CGPA', 'Chance_of_Admit']].corr(method='pearson')

In [26]:
corr2

,GRE,TOEFL,CGPA,Chance_of_Admit
GRE,1.000000,0.835977,0.833060,0.802610
TOEFL,0.835977,1.000000,0.828417,0.791594
CGPA,0.833060,0.828417,1.000000,0.873289
Chance_of_Admit,0.802610,0.791594,0.873289,1.000000


In [29]:
corr2['Chance_of_Admit'][:-1].max()

0.8732890993553003

### 답
0.873

## 2. GRE 점수의 평균 이상, 미만인 그룹의 CGPA 평균 차이 검정
- 등분산 가정, 양측 검정
- 검정통계량을 소수점 셋째 자리에서 반올림하여 소수점 둘째 자리까지

In [30]:
gre_mean = df2['GRE'].mean()

In [34]:
df2_ge = df2[df2['GRE'] >= gre_mean]['CGPA']
df2_le = df2[df2['GRE'] < gre_mean]['CGPA']

In [39]:
from scipy.stats import ttest_ind

t, p = ttest_ind(df2_ge, df2_le, equal_var=True, alternative='two-sided')
t, p
print(f'검정통계량 : {t:.4f}, p-value : {p:.4f}')

검정통계량 : 19.4433, p-value : 0.0000


### 답
19.44

### 참고

In [60]:
df2_add = df2_orig.copy()
df2_add['GRE_gr'] = 'under'
df2_add.loc[df2_add['GRE'] >= gre_mean, 'GRE_gr'] = 'over'

In [62]:
df2_add.head(3)

,Serial_No,GRE,TOEFL,University_Rating,SOP,LOR,CGPA,Research,Chance_of_Admit,GRE_gr
0,1,337,118,4,4.5,4.5,9.65,1,0.92,over
1,2,324,107,4,4.0,4.5,8.87,1,0.76,over
2,3,316,104,3,3.0,3.5,8.00,1,0.72,under


## 3. 로지스틱 회귀 분석
- Chance_of_Admit > 0.5면 합격, 아니면 불합격
- Serial_No, Label 제외
- seed=123
- 로지스틱 회귀계수 절댓값이 가장 큰 변수와 그 값을 기술
- 반올림하여 소수점 둘째자리까지

In [40]:
df2['Admit'] = df2['Chance_of_Admit'].apply(lambda x:1 if x > 0.5 else 0)
df2.head(3)

,Serial_No,GRE,TOEFL,University_Rating,SOP,LOR,CGPA,Research,Chance_of_Admit,Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92,1
1,2,324,107,4,4.0,4.5,8.87,1,0.76,1
2,3,316,104,3,3.0,3.5,8.00,1,0.72,1


In [41]:
X_features = df2.drop(columns=['Serial_No', 'Chance_of_Admit', 'Admit'])
y_label = df2['Admit']

In [66]:
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression(random_state=123)
lr_clf.fit(X_features, y_label)

C:\Users\master\anaconda3\envs\multi\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(random_state=123)

In [67]:
coef_list = []
for var, coef in zip(X_features.columns, lr_clf.coef_[0]):
    coef_list.append((var, abs(coef)))

coef_list.sort(key=lambda x:x[1], reverse=True)

In [68]:
coef_list

[('CGPA', 2.1209474063809477),
 ('LOR', 1.0257030758582966),
 ('Research', 0.5135235309968297),
 ('SOP', 0.207305277499899),
 ('TOEFL', 0.10476061517172451),
 ('GRE', 0.0897434566374038),
 ('University_Rating', 0.0003943235267222636)]

In [69]:
df2_coef = pd.DataFrame(lr_clf.coef_, columns=X_features.columns)
df2_coef

,GRE,TOEFL,University_Rating,SOP,LOR,CGPA,Research
0,-0.089743,0.104761,0.000394,-0.207305,1.025703,2.120947,0.513524


### 답
CGPA, 2.12

### 참고

In [ ]:
df2['Admit'] = np.where(df2['Chance_of_Admit'] > 0.5, 1, 0)

# 문제 08 유형
- DataSet_08.csv 이용

In [58]:
import pandas as pd

df3_orig = pd.read_csv('datasets/Dataset_08.csv', encoding='utf8')
df3 = df3_orig.copy()
df3.head(3)

,RandD_Spend,Administration,Marketing_Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39


In [59]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RandD_Spend      50 non-null     float64
 1   Administration   50 non-null     float64
 2   Marketing_Spend  50 non-null     float64
 3   State            50 non-null     object 
 4   Profit           50 non-null     float64
dtypes: float64(4), object(1)
memory usage: 2.1+ KB


## 1. 각 주(State)별 데이터 구성비
- 알파벳 순으로 소수점 둘째 자리까지

In [72]:
df3_pivot = pd.pivot_table(data=df3, index='State', aggfunc='count')

In [73]:
df3_pivot

,Administration,Marketing_Spend,Profit,RandD_Spend
State,,,,
California,17,17,17,17
Florida,16,16,16,16
New York,17,17,17,17


In [74]:
df3_state = df3.groupby('State').size() / len(df3)
df3_state

State
California    0.34
Florida       0.32
New York      0.34
dtype: float64

### 답
0.34, 0.32, 0.34

## 2. 주별 이익 평균과 큰 주와 작은 주의 차이
- 소수점 이하는 버리고 정수 부분만

In [75]:
df3_state_mean = df3.groupby('State').mean()

In [76]:
df3_state_mean

,RandD_Spend,Administration,Marketing_Spend,Profit
State,,,,
California,64657.674118,120746.507059,182540.929412,103905.175294
Florida,80724.016250,121768.897500,247323.551250,118774.024375
New York,76195.062353,121543.470588,205346.015882,113756.446471


In [77]:
df3_state_mean['Profit'].max() - df3_state_mean['Profit'].min()

14868.84908088237

### 답
14868

## 3. Profit을 주별로 예측하는 회귀 모형
- RandD_Spend, Administration, Marketing_Spend를 독립변수
- 학습 오차 MAPE 기준 가장 낮은 오차를 보여주는 주와 그 값
- 반올림하여 소수점 둘째 자리까지

In [90]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error
import numpy as np

for state in df3['State'].unique():
    X = df3[df3['State']==state].drop(columns=['State', 'Profit'])
    y = df3[df3['State']==state]['Profit']
    lr = LinearRegression()
    lr.fit(X, y)
    pred = lr.predict(X)
    mape = mean_absolute_percentage_error(y, pred) * 100
    print(state, 'MAPE :', np.round(mape, 3))

New York MAPE : 8.524
California MAPE : 15.228
Florida MAPE : 5.707


### 답
California, 15.29

# 문제 09 유형
- DataSet_09.csv 이용

In [128]:
df4 = pd.read_csv('datasets/Dataset_09.csv', encoding='utf8')
df4.head(3)

,satisfaction,Gender,Age,Customer_Type,Class,Flight_Distance,Seat_comfort,Food_and_drink,Inflight_wifi_service,Inflight_entertainment,Onboard_service,Leg_room_service,Baggage_handling,Cleanliness,Departure_Delay_in_Minutes,Arrival_Delay_in_Minutes
0,satisfied,Female,19.0,Loyal Customer,Eco,1981.0,4.0,4.0,1.0,4.0,4.0,4.0,4.0,4.0,3.0,28.0
1,dissatisfied,Female,64.0,Loyal Customer,Eco,2847.0,1.0,1.0,1.0,2.0,5.0,3.0,4.0,4.0,111.0,133.0
2,dissatisfied,Male,55.0,Loyal Customer,Eco,3790.0,1.0,1.0,2.0,1.0,4.0,4.0,4.0,4.0,2.0,15.0


## 1. 결측값 개수

In [129]:
import numpy as np

df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   satisfaction                2000 non-null   object 
 1   Gender                      2000 non-null   object 
 2   Age                         2000 non-null   float64
 3   Customer_Type               2000 non-null   object 
 4   Class                       2000 non-null   object 
 5   Flight_Distance             2000 non-null   float64
 6   Seat_comfort                2000 non-null   float64
 7   Food_and_drink              2000 non-null   float64
 8   Inflight_wifi_service       2000 non-null   float64
 9   Inflight_entertainment      2000 non-null   float64
 10  Onboard_service             2000 non-null   float64
 11  Leg_room_service            2000 non-null   float64
 12  Baggage_handling            2000 non-null   float64
 13  Cleanliness                 2000 

In [130]:
df4.isna().sum()

satisfaction                  0
Gender                        0
Age                           0
Customer_Type                 0
Class                         0
Flight_Distance               0
Seat_comfort                  0
Food_and_drink                0
Inflight_wifi_service         0
Inflight_entertainment        0
Onboard_service               0
Leg_room_service              0
Baggage_handling              0
Cleanliness                   0
Departure_Delay_in_Minutes    0
Arrival_Delay_in_Minutes      5
dtype: int64

### 답
5

## 2. 카이제곱 독립성 검정
- 결측값 제거
- Age_gr 파생변수 생성(10, 20, 30, 40, 50, 60)
- Age_gr, Gender, Customer_Type, Class가 satisfaction에 영향이 있는지
- 연관성이 있는 변수의 검정통계량을 정수 부분만

In [131]:
df4_dropna = df4.dropna()

In [132]:
def age_gr(age):
    if age <=20:
        return 10
    elif age <= 30:
        return 20
    elif age <= 40:
        return 30
    elif age <= 50:
        return 40
    elif age <= 60:
        return 50
    else:
        return 60

In [134]:
from scipy.stats import chi2_contingency

df4_dropna['Age_gr'] = df4_dropna['Age'].apply(lambda x:age_gr(x))

var_list = ['Age_gr', 'Gender', 'Customer_Type', 'Class']

for var in var_list:
    pt = pd.pivot_table(data=df4_dropna, index=var, columns='satisfaction',
                       aggfunc='count')
    chi2, p, _, _ = chi2_contingency(pt)
    print(f'{var}의 검정통계량 : {chi2:.1f}, p-value : {p:.3f}')

Age_gr의 검정통계량 : 34.3, p-value : 1.000
Gender의 검정통계량 : 16045.8, p-value : 0.000
Customer_Type의 검정통계량 : 15.1, p-value : 0.984
Class의 검정통계량 : 4.7, p-value : 1.000


C:\Users\master\anaconda3\envs\multi\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


### 답
16045

## 3. 로지스틱 회귀 분석
- 결측치 제거
- train:test = 7:3, seed=123
- Flight_Distance, Seat_comfort, Food_and_drink, Inflight_wifi_service, 
  Inflight_entertainment,Onboard_service, Leg_room_service, Baggage_handling,
  Cleanliness, Departure_Delay_in_Minutes, Arrival_Delay_in_Minutes
- 고객만족도를 label
- seed=123
- 예측 정확도 측정, dissatisfied의 f1 score를 소수점 넷째 자리에서 반올림하여 셋째 자리까지

In [136]:
X = df4_dropna[['Flight_Distance', 'Seat_comfort', 'Food_and_drink', 
                'Inflight_wifi_service', 'Inflight_entertainment',
                'Onboard_service', 'Leg_room_service', 'Baggage_handling', 
                'Cleanliness', 'Departure_Delay_in_Minutes', 
                'Arrival_Delay_in_Minutes']]
y = df4_dropna['satisfaction']

In [138]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)

lr_clf = LogisticRegression(random_state=123)
lr_clf.fit(X_train, y_train)

pred = lr_clf.predict(X_test)
f1 = f1_score(y_test, pred, pos_label='dissatisfied')
print(f1)

0.7302752293577981


C:\Users\master\anaconda3\envs\multi\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


### 답
0.730

# 문제 10 유형
- DataSet_10.csv 이용

In [111]:
df5_orig = pd.read_csv('datasets/Dataset_10.csv', encoding='utf8')
df5 = df5_orig.iloc[:, :6]
df5.head(3)

,model,engine_power,age_in_days,km,previous_owners,price
0,lounge,51,882,25000,1,8900
1,pop,51,1186,32500,1,8800
2,sport,74,4658,142228,1,4200


### 참고

In [112]:
df5_dropna = df5_orig.dropna(axis=1, how='all')
df5_dropna.head(3)

,model,engine_power,age_in_days,km,previous_owners,price
0,lounge,51,882,25000,1,8900
1,pop,51,1186,32500,1,8800
2,sport,74,4658,142228,1,4200


## 1. 모델별 하루 평균 운행 거리 산출
- 소유자가 1, 엔진파워가 51인 차
- 가장 낮은 값 / 가장 큰 값 비율을 반올림하여 소수점 둘째자리까지
- 모델별 평균 > 일평균 > 최대최소 비율 계산 순으로 

In [113]:
df5_daily = df5[(df5['previous_owners'] == 1) & (df5['engine_power']==51)]
df5_daily.head(3)

,model,engine_power,age_in_days,km,previous_owners,price
0,lounge,51,882,25000,1,8900
1,pop,51,1186,32500,1,8800
3,lounge,51,2739,160000,1,6000


In [114]:
df5_daily = df5_daily.groupby('model').mean()
df5_daily

,engine_power,age_in_days,km,previous_owners,price
model,,,,,
lounge,51.0,1343.450366,44123.670846,1.0,9072.819227
pop,51.0,1800.509804,58097.532680,1.0,8118.091503
sport,51.0,3031.387755,96460.979592,1.0,6719.959184


In [115]:
df5_daily['daily_average'] = df5_daily['km'] / df5_daily['age_in_days']
df5_daily

,engine_power,age_in_days,km,previous_owners,price,daily_average
model,,,,,,
lounge,51.0,1343.450366,44123.670846,1.0,9072.819227,32.843544
pop,51.0,1800.509804,58097.532680,1.0,8118.091503,32.267268
sport,51.0,3031.387755,96460.979592,1.0,6719.959184,31.820733


In [116]:
df5_ratio = df5_daily['daily_average'].min() / df5_daily['daily_average'].max()
print(df5_ratio)

0.9688580804724013


### 답
0.97

### 참고

In [119]:
df5_daily2 = df5[(df5['previous_owners'] == 1) & (df5['engine_power']==51)]
df5_daily2 = pd.pivot_table(data=df5_daily2, index='model', values=['age_in_days', 'km'],
                           aggfunc=np.mean)
df5_daily2

,age_in_days,km
model,,
lounge,1343.450366,44123.670846
pop,1800.509804,58097.532680
sport,3031.387755,96460.979592


In [120]:
df5_daily2['km'] / df5_daily2['age_in_days']

model
lounge    32.843544
pop       32.267268
sport     31.820733
dtype: float64

## 2. 운행 일수에 따른 운행 거리
- 1번 문제에서 가장 큰 값을 가지고 있던 모델과 낮은 모델 간 운행 일수 대비 운행 거리 평균에 대한 통계 검정
- p-value 소수점 세자리 이하는 버리고 둘째자리까지
- 기각 여부를 Y/N
- 등분산 가정

In [122]:
df5_model_daily = df5.copy()

df5_model_daily['km_per_day'] = df5_model_daily['km'] / df5_model_daily['age_in_days']
lounge = df5_model_daily[df5_model_daily['model']=='lounge']['km_per_day']
sport = df5_model_daily[df5_model_daily['model']=='sport']['km_per_day']

In [123]:
from scipy.stats import ttest_ind

t, p = ttest_ind(lounge, sport, equal_var=True)
print(t, p)

1.5054268543179226 0.13248244438755083


### 답
0.13, N

## 3. 선형회귀분석
- 독립변수 engine_power, age_in_days, km
- 종속변수 price
- model pop, engine_power 51, age_in_days 400, km 9500, previous_owners 2에 대한 예측 가격을 정수만

In [125]:
pop = df5[df5['model'] == 'pop']
X = pop.drop(columns=['model', 'price'])
y = pop['price']

X.shape

(358, 4)

In [127]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X, y)
pred = lr.predict([[51, 400, 9500, 2]])
print(pred)

[10252.24314359]


### 답
10252